In [3]:
import requests
from bs4 import BeautifulSoup
from lxml import html
import json
import re
import csv
import pandas as pd
from collections import Counter
from tqdm import tqdm

from wiktionaryparser import WiktionaryParser
parser = WiktionaryParser()

In [1]:
url = 'https://en.wiktionary.org/wiki/Category:Spanish_idioms'
dict_of_links = {}
while True:
    page = requests.get(url)
    html = page.text
    soup = BeautifulSoup(html)
    for item in soup.select("div[class='mw-category-generated'] a[href^='/wiki/']"):
        if re.search('(?<=>)(.*)(?=<)', str(item)).group(0) == 'Synonymes de langues en français':
            pass
        else:
            collocation = re.search('(?<=>)(.*)(?=<)', str(item)).group(0)
            link = 'https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0)
            dict_of_links[collocation] = link
        #     print('https://en.wiktionary.org'+re.search('(?<=href=")([^"]*)', str(item)).group(0))
    next_page_available = 0
    for i in soup.find_all('a'):
        if i.string == 'next page':
            url='https://en.wiktionary.org/'+i['href']
            next_page_available = 1
            break
    if not next_page_available:
        break
dict_of_links

In [6]:
del dict_of_links['Spanish rhetorical questions']
del dict_of_links['Spanish verbs with lexical clitics']

In [7]:
len(dict_of_links)

2535

In [2]:
list_for_file = []
for collocation in dict_of_links:
    try:
        collocation_info = parser.fetch(collocation, 'spanish')
        if collocation_info!=[]:
            if collocation_info[0]['definitions']!=[]:
                for i in range(len(collocation_info[0]['definitions'])):
                    list_for_collocation = [collocation]
                    if 'partOfSpeech' in collocation_info[0]['definitions'][i]:
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['partOfSpeech'])
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    else:
                        list_for_collocation.append('')
                        list_for_collocation.append(collocation_info[0]['definitions'][i]['text'][1:])
                    list_for_collocation.append(dict_of_links[collocation])
                    list_for_file.append(list_for_collocation)
    except AttributeError:
        pass
list_for_file

In [12]:
with open('Spanish_idioms.tsv', 'w', newline='') as record_file:
    tsv_writer = csv.writer(record_file, delimiter='\t', lineterminator='\n')
    tsv_writer.writerow(['Collocation', 'Part_of_speech', 'Definition', 'Link'])
    for collocation_info in list_for_file:
        tsv_writer.writerow([collocation_info[0], collocation_info[1], collocation_info[2],\
                             dict_of_links[collocation_info[0]]])

In [ ]:
df = pd.read_csv("Spanish_idioms.tsv",sep='\t')
def left_align(df):
    left_aligned_df = df.style.set_properties(**{'text-align': 'left'})
    left_aligned_df = left_aligned_df.set_table_styles(
        [dict(selector='th', props=[('text-align', 'left')])]
    )
    return left_aligned_df
pd.options.display.max_colwidth=100
pd.options.display.max_rows=50
# display(left_align(df[['Collocation','Part_of_speech','Definition']].head(500)))
display(left_align(df))

In [14]:
list_of_literal_meanings = []
for collocation in tqdm(df['Collocation']):
    collocation_info = parser.fetch(collocation, 'spanish')
    if collocation_info!=[]:
        if collocation_info[0]['etymology']!=[]:
            if not re.search('(?<=Literally, ")([^"]*)', collocation_info[0]['etymology']):
                list_of_literal_meanings.append('')
            else:
                list_of_literal_meanings.append(re.search('(?<=Literally, ")([^"]*)', \
                                                      collocation_info[0]['etymology']).group(0))

100%|███████████████████████████████████████| 2568/2568 [21:34<00:00,  1.98it/s]


In [15]:
len(list_of_literal_meanings)

2568

In [17]:
df.head()

,Collocation,Part_of_speech,Definition,Link
0,a base de bien,adverb,"['(Spain, idiomatic) loads; galore']",https://en.wiktionary.org/wiki/a_base_de_bien
1,a bote pronto,adverb,"[""(idiomatic) off the top of one's head""]",https://en.wiktionary.org/wiki/a_bote_pronto
2,a brazo partido,adverb,['(idiomatic) tooth and nail'],https://en.wiktionary.org/wiki/a_brazo_partido
3,a buen recaudo,adverb,['(idiomatic) safe and sound'],https://en.wiktionary.org/wiki/a_buen_recaudo
4,a buen seguro,adverb,"['(idiomatic) surely, for sure, no doubt']",https://en.wiktionary.org/wiki/a_buen_seguro


In [18]:
df.insert(2, 'Literal meaning', list_of_literal_meanings)
df.head()

,Collocation,Part_of_speech,Literal meaning,Definition,Link
0,a base de bien,adverb,,"['(Spain, idiomatic) loads; galore']",https://en.wiktionary.org/wiki/a_base_de_bien
1,a bote pronto,adverb,,"[""(idiomatic) off the top of one's head""]",https://en.wiktionary.org/wiki/a_bote_pronto
2,a brazo partido,adverb,,['(idiomatic) tooth and nail'],https://en.wiktionary.org/wiki/a_brazo_partido
3,a buen recaudo,adverb,,['(idiomatic) safe and sound'],https://en.wiktionary.org/wiki/a_buen_recaudo
4,a buen seguro,adverb,,"['(idiomatic) surely, for sure, no doubt']",https://en.wiktionary.org/wiki/a_buen_seguro


In [19]:
df.to_csv("spanish_idioms_wLM.tsv", sep = "\t", index=False)

In [7]:
len(df)

2568